In [0]:
from pyspark.sql.functions import col, to_date

# Lê os dados brutos
def read_bronze():
    df_bronze = spark.read.json('/Volumes/raw/ipca_bronze/ipca_raw')
    return df_bronze

# Cria o schema (caso ainda não exista)
def create_schema():
    spark.sql("CREATE SCHEMA IF NOT EXISTS silver")
    return True

# Dropar a tabela caso já exista com schema antigo
def drop_old():
    spark.sql("DROP TABLE IF EXISTS silver.ipca_silver")
    return True

# Trata e onverte os tipos de dados 
def clean_and_convert(df_bronze):
    df_silver = (df_bronze
        .withColumn("data", to_date(col("data"), "dd/MM/yyyy"))
        .withColumn("valor", col("valor").cast("double"))  
        .dropDuplicates()        
        .na.drop()                                        
    )
    return df_silver

# Salva o dataframe tratado como tabela delta na camada silver
def save_as_delta(df_silver):
    (df_silver.distinct()
        .coalesce(1)
        .write
        .format('delta')
        .mode('overwrite')
        .saveAsTable('silver.ipca_silver')
    )

# Executa o passo a passo para salvar os dados
def main():
    df_bronze = read_bronze()
    create_schema()
    drop_old()
    df_silver = clean_and_convert(df_bronze)
    save_as_delta(df_silver)
    return True
main()